In [1]:
import pandas as pd
from pyviper._load._load_regulators import load_TFs, load_coTFs, load_sig, load_surf
from pyviper._load._load_translate import load_human2mouse

In [2]:
tfs = load_TFs(species="human")
cotfs = load_coTFs(species="human")
sigs = load_sig(species="human")
surfs = load_surf(species="human")

In [3]:
gene_names = load_human2mouse()

In [4]:
gene_names

,human_symbol,mouse_symbol,human_ensembl,mouse_ensembl,human_entrez,mouse_entrez
0,A1BG,A1bg,ENSG00000121410,ENSMUSG00000022347,1,117586
1,A1CF,A1cf,ENSG00000148584,ENSMUSG00000052595,29974,69865
2,A2M,A2m,ENSG00000175899,ENSMUSG00000030111,2,232345
3,A3GALT2,A3galt2,ENSG00000184389,ENSMUSG00000028794,127550,215493
4,A4GALT,A4galt,ENSG00000128274,ENSMUSG00000047878,53947,239559
...,...,...,...,...,...,...
28789,ZYG11B,Zyg11b,ENSG00000162378,ENSMUSG00000034636,79699,414872
28790,ZYX,Zyx,ENSG00000159840,ENSMUSG00000029860,7791,22793
28791,ZYX,Zyx,ENSG00000285443,ENSMUSG00000029860,7791,22793
28792,ZZEF1,Zzef1,ENSG00000074755,ENSMUSG00000055670,23140,195018


In [5]:
symbol2ensembl = gene_names.set_index("human_symbol")["human_ensembl"].to_dict()

In [6]:
regulators = pd.Series([symbol2ensembl.get(reg) for reg in tfs + cotfs])

In [7]:
# check nas
regulators.isna().mean()

0.12433754586220953

In [8]:
regulators = regulators.pipe(lambda x: x[~x.isna()])

In [9]:
regulators.isna().mean()

0.0

In [10]:
regulators.to_csv("/burg/pmg/users/rc3686/data/regulators.txt", header=False, index=False)

In [11]:
len(regulators)

2148

### Making global gene to node file

In [6]:
sample_aracne_out = '/burg/pmg/users/rc3686/data/cellxgene/data/cell_type_005/dendritic_cell'
ranks = pd.read_csv(sample_aracne_out + "/rank_raw.csv", index_col=None, nrows=10)

In [9]:
gene_list = ranks.columns.tolist()
gene_list[:10]

['ENSG00000121410',
 'ENSG00000148584',
 'ENSG00000175899',
 'ENSG00000166535',
 'ENSG00000184389',
 'ENSG00000128274',
 'ENSG00000118017',
 'ENSG00000081760',
 'ENSG00000114771',
 'ENSG00000188984']

In [10]:
full_gene_list = ["<PAD>", "<MASK>"] + sorted(gene_list)

In [11]:
global_gene_to_node = pd.DataFrame({
    "gene_name": full_gene_list,
    "idx": range(len(full_gene_list))
})

In [12]:
global_gene_to_node

,gene_name,idx
0,<PAD>,0
1,<MASK>,1
2,ENSG00000000003,2
3,ENSG00000000005,3
4,ENSG00000000419,4
...,...,...
19241,ENSG00000288701,19241
19242,ENSG00000288702,19242
19243,ENSG00000288705,19243
19244,ENSG00000288709,19244


In [13]:
gene_to_node_file = '/burg/pmg/collab/scGraphLLM/data/cellxgene_gene2index.csv'

In [14]:
global_gene_to_node.to_csv(
    gene_to_node_file, index=False, header=True
)